In [ ]:
library(DESeq2)
library(EnhancedVolcano)
library(ggplot2)

In [ ]:
tnk_gene_exp <- read.csv("tnk_celltype_pooled_expression_by_sample.csv",row.names=1)
metadata <- read.csv("tnk_celltype_pooled_metadata.csv", row.names=1)


In [ ]:
# dropna
tnk_gene_exp <- na.omit(tnk_gene_exp)
tnk_gene_exp <- t(tnk_gene_exp)


In [ ]:
metadata = metadata[colnames(tnk_gene_exp),]


In [ ]:


# remove genes not expressed in > 20% of samples
tnk_gene_exp<-tnk_gene_exp[rowCounts(tnk_gene_exp>0, value=TRUE) >= 0.2 * dim(tnk_gene_exp)[2],]

# remove sample/celltypes with < 20 cells
metadata <- metadata[metadata$Cell_Number > 10,]

tnk_gene_exp = tnk_gene_exp[,rownames(metadata)]


In [ ]:
metadata$M.Number <- factor(metadata$M.Number)

In [ ]:
# to make the design matrix work
metadata$monkey_condition_categories <- factor(metadata$monkey_condition_categories)

In [ ]:
# to get celltype vs celltype differential expression
dds <- DESeqDataSetFromMatrix(countData = tnk_gene_exp, colData = metadata, design = ~ factor(M.Number)+celltype)


In [ ]:
metadata$is.T1T17 <- metadata$NK.T.clusters=="T1T17" # this is to get T1T17 marker genes

In [ ]:
# getting T1T17 differences between treatments
# interaction gives the difference between condition effects here, that is not really what we want
# we want the condition effect that is specific to the T1T17 cells
# well maybe that is what we want, we want genes that change between treatments in T1T17 cells but not necessarily in all cells
dds <- DESeqDataSetFromMatrix(countData = tnk_gene_exp, colData = metadata, design = ~ factor(monkey_condition_categories)+treatment+is.T1T17+treatment:is.T1T17)
dds$treatment <- relevel(dds$treatment, ref = "IgG")
dds_wald <- DESeq(dds,test="Wald")
#dds_lrt <- DESeq(dds, test="LRT", reduced = ~ factor(monkey_condition_categories))#+treatment+is.T1T17


In [ ]:
resultsNames(dds_wald)


In [ ]:
res<-results(dds_wald,name='treatmentCD8a.is.T1T17TRUE')
ressub <-subset(res,res$padj < 0.05)
head(ressub[order(ressub$log2FoldChange,decreasing = TRUE),],20)

In [ ]:
res<-results(dds_wald,name='treatmentCD8b.is.T1T17TRUE')
ressub <-subset(res,res$padj < 0.05)
head(ressub[order(ressub$log2FoldChange,decreasing = TRUE),],20)

In [ ]:
#make a group for condition+isT1T17 and directly contrast those instead of doing interaction terms
dds$group <- factor(paste0(dds$treatment, dds$is.T1T17))
design(dds) <- ~ factor(monkey_condition_categories)+group
dds_wald_2 <- DESeq(dds, test="Wald")

In [ ]:
unique(dds$group)

In [ ]:
resultsNames(dds_wald_2)

In [ ]:
res<-results(dds_wald_2,contrast=c("group","CD8bTRUE","IgGTRUE"))

In [ ]:
# results for making a group instead of interaction terms
ressub <-subset(res,res$padj < 0.05) 
ressub <-subset(ressub,ressub$baseMean > 20)
head(ressub[order(ressub$log2FoldChange,decreasing = TRUE),],20)

In [ ]:
# results for making a group instead of interaction terms
res<-results(dds_wald_2,contrast=c("group","CD8aTRUE","IgGTRUE"))
ressub <-subset(res,res$padj < 0.05) 
#ressub <-subset(ressub,ressub$baseMean > 20)
head(ressub[order(ressub$log2FoldChange,decreasing = FALSE),],20)

In [ ]:
# looking at just the T1T17 cluster to do differential expression between conditions instead of dealing with interaction terms

dds_t1t17 <- DESeqDataSetFromMatrix(countData = tnk_gene_exp[,rownames(metadata[metadata$is.T1T17,])], colData = metadata[metadata$is.T1T17,], design = ~ factor(monkey_condition_categories)+treatment)


In [ ]:
resultsNames(dds_t1t17_lrt)

In [ ]:
res<-results(dds_t1t17_lrt,contrast=c('treatment','IgG','CD8b'))

In [ ]:
ressub <-subset(res,res$padj < 0.05)
ressub <-subset(ressub,ressub$baseMean > 20)
head(ressub[order(ressub$log2FoldChange,decreasing = TRUE),],15)

In [ ]:
tail(ressub[order(ressub$log2FoldChange,decreasing = TRUE),],10)

In [ ]:
# what if monkey is not included in the design formula?
dds_t1t17 <- DESeqDataSetFromMatrix(countData = tnk_gene_exp[,rownames(metadata[metadata$is.T1T17,])], colData = metadata[metadata$is.T1T17,], design = ~ treatment)
dds_t1t17_lrt <- DESeq(dds_t1t17, test="LRT", reduced = ~ 1)

In [ ]:
res<-results(dds_t1t17_lrt,contrast=c('treatment','IgG','CD8b'))

In [ ]:
ressub <-subset(res,res$padj < 0.05)
ressub <-subset(ressub,ressub$baseMean > 20)
head(ressub[order(ressub$log2FoldChange,decreasing = TRUE),],15)

In [ ]:
dds_t1t17 <- DESeqDataSetFromMatrix(countData = tnk_gene_exp[,rownames(metadata[metadata$is.T1T17,])], colData = metadata[metadata$is.T1T17,], design = ~factor(monkey_condition_categories)+ treatment)
dds_t1t17_wald <- DESeq(dds_t1t17, test="Wald")

In [ ]:
res<-results(dds_t1t17_wald,contrast=c('treatment','CD8a','IgG'))
ressub <-subset(res,res$padj < 0.05)
ressub <-subset(ressub,ressub$baseMean > 20)
head(ressub[order(ressub$log2FoldChange,decreasing = TRUE),],15)
write.csv(res, "T1T17_CD8avsIgG.csv",quote=FALSE)
options(repr.plot.width = 10, repr.plot.height = 15, repr.plot.res = 100)

EnhancedVolcano(res,
    lab = rownames(res),
    x = 'log2FoldChange',
    y = 'pvalue',
    title = 'T1T17 CD8a vs IgG',
    #pCutoff = 10e-32,
    FCcutoff = 1.3,
    pointSize = 3.0,
    labSize = 6.0,
               xlim=c(-10,8))

In [ ]:
res<-results(dds_t1t17_wald,contrast=c('treatment','CD8b','IgG'))
ressub <-subset(res,res$padj < 0.05)
ressub <-subset(ressub,ressub$baseMean > 20)
head(ressub[order(ressub$log2FoldChange,decreasing = TRUE),],15)
write.csv(res, "T1T17_CD8bvsIgG.csv",quote=FALSE)
EnhancedVolcano(res,
    lab = rownames(res),
    x = 'log2FoldChange',
    y = 'pvalue',
    title = 'T1T17 CD8b vs IgG',
    #pCutoff = 10e-32,
    FCcutoff = 1.3,
    pointSize = 3.0,
    labSize = 6.0,
               xlim=c(-10,8))

In [ ]:
# pca on the cytotoxic subsets

In [ ]:
dds_cytotoxic_subclusts <- DESeqDataSetFromMatrix(countData = tnk_gene_exp[,rownames(metadata[metadata$NK.T.clusters == "Cytotoxic T/NK",])], colData = metadata[metadata$NK.T.clusters == "Cytotoxic T/NK",], design = ~ factor(monkey_condition_categories)+treatment)


In [ ]:
vst_cytotoxic_subclusts <- vst(dds_cytotoxic_subclusts, blind=FALSE)

In [ ]:
plotPCA(vst_cytotoxic_subclusts, intgroup="NK.T.Subclusters")


In [ ]:
plotPCA(vst_cytotoxic_subclusts, intgroup="treatment")


In [ ]:
plotPCA(vst_cytotoxic_subclusts, intgroup="M.Number")


In [ ]:
colData(vst_cytotoxic_subclusts)$GZMB<-assay(vst_cytotoxic_subclusts["GZMB",])
pcaData <- plotPCA(vst_cytotoxic_subclusts, intgroup="GZMB", returnData=TRUE)
percentVar <- round(100 * attr(pcaData, "percentVar"))
ggplot(pcaData, aes(PC1, PC2, color=GZMB)) +
  geom_point(size=3) +
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance")) +  theme_classic()+
  coord_fixed() +scale_colour_gradient(low = "gold", high = "red2")

In [ ]:
colData(vst_cytotoxic_subclusts)$CD4<-assay(vst_cytotoxic_subclusts["CD4",])
pcaData <- plotPCA(vst_cytotoxic_subclusts, intgroup="CD4", returnData=TRUE)
percentVar <- round(100 * attr(pcaData, "percentVar"))
ggplot(pcaData, aes(PC1, PC2, color=CD4)) +
  geom_point(size=3) +
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance")) +  theme_classic()+
  coord_fixed() +scale_colour_gradient(low = "gold", high = "red2")

In [ ]:
colData(vst_cytotoxic_subclusts)$CD8A<-assay(vst_cytotoxic_subclusts["CD8A",])
pcaData <- plotPCA(vst_cytotoxic_subclusts, intgroup="CD8A", returnData=TRUE)
percentVar <- round(100 * attr(pcaData, "percentVar"))
ggplot(pcaData, aes(PC1, PC2, color=CD8A)) +
  geom_point(size=3) +
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance")) +  theme_classic()+
  coord_fixed() +scale_colour_gradient(low = "gold", high = "red2")

In [ ]:
pca <- prcomp(t(assay(vst_cytotoxic_subclusts)))
loadings <- as.data.frame(pca$rotation)

In [ ]:
loadings[order(loadings$PC1,decreasing=TRUE),]

In [ ]:
loadings[order(loadings$PC1),]

In [ ]:
pc$loadings

In [ ]:
colData(vst_cytotoxic_subclusts)$GZMK<-assay(vst_cytotoxic_subclusts["GZMK",])
pcaData <- plotPCA(vst_cytotoxic_subclusts, intgroup="GZMK", returnData=TRUE)
percentVar <- round(100 * attr(pcaData, "percentVar"))
ggplot(pcaData, aes(PC1, PC2, color=GZMK)) +
  geom_point(size=3) +
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance")) +  theme_classic()+
  coord_fixed() +scale_colour_gradient(low = "gold", high = "red2")
#+scale_color_manual(values=c("#FF00FF","#4F97A3","#5fe8ed","#6593f5","#fda89f","#b97ffa","#48260D","#95c8d8","#FF2400","#0552f5","#2e6669","#b0ebc1","#7f19e6","#3f704d","#Df5386","#98FB98","#88807B"))



In [ ]:
library(ggplot2)

In [ ]:
# one cluster, differential expression between conditions (Cytotoxic 4)

metadata$is.innatelike <- metadata$NK.T.Subclusters=="Cytotoxic 4"#"innate-like cytotoxic"
dds_innatelike <- DESeqDataSetFromMatrix(countData = tnk_gene_exp[,rownames(metadata[metadata$is.innatelike,])], colData = metadata[metadata$is.innatelike,], design = ~ factor(monkey_condition_categories)+treatment)
vst_innatelike <- vst(dds_innatelike, blind=FALSE)

pcaData <- plotPCA(vst_innatelike, intgroup=c("Gran.State"), returnData=TRUE)
percentVar <- round(100 * attr(pcaData, "percentVar"))
ggplot(pcaData, aes(PC1, PC2, color=Gran.State)) +
  geom_point(size=3) +
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance")) +  theme_classic()+
  coord_fixed()


pcaData <- plotPCA(vst_innatelike, intgroup=c("treatment"), returnData=TRUE)
percentVar <- round(100 * attr(pcaData, "percentVar"))
ggplot(pcaData, aes(PC1, PC2, color=treatment)) +
  geom_point(size=3) +
  xlab(paste0("PC1: ",percentVar[1],"% variance")) +
  ylab(paste0("PC2: ",percentVar[2],"% variance")) +  theme_classic()+
  coord_fixed()


dds_innatelike_wald <- DESeq(dds_innatelike,test="Wald")

res <- results(dds_innatelike_wald, contrast = c("treatment","CD8a","IgG"))
options(repr.plot.width = 10, repr.plot.height = 15, repr.plot.res = 100)

EnhancedVolcano(res,
    lab = rownames(res),
    x = 'log2FoldChange',
    y = 'pvalue',
    title = 'Innate-like Cytotoxic aCD8a vs IgG',
    #pCutoff = 10e-32,
    FCcutoff = 1.3,
    pointSize = 3.0,
    labSize = 6.0,
               xlim=c(-15,7))



In [ ]:
ressub <-subset(res,res$padj < 0.05)
#ressub <-subset(ressub,ressub$baseMean > 20)
rownames(head(ressub[order(ressub$log2FoldChange,decreasing = TRUE),],20))

In [ ]:
rownames(tail(ressub[order(ressub$log2FoldChange,decreasing = TRUE),],20))

In [ ]:
# compare clusters between each other
dds <- DESeqDataSetFromMatrix(countData = tnk_gene_exp, colData = metadata, design = ~ factor(M.Number)+NK.T.Subclusters)
dds$treatment <- relevel(dds$treatment, ref = "IgG")
dds_wald <- DESeq(dds, test="Wald")

In [ ]:
res_GZMK_subsets <- results(dds_wald, contrast=c("NK.T.Subclusters","Cytotoxic 1","Cytotoxic 2"))
ressub <-subset(res_GZMK_subsets,res$padj < 0.05)
ressub <-subset(ressub,ressub$baseMean > 20)
head(ressub[order(ressub$log2FoldChange,decreasing = TRUE),],10)
write.csv(res_GZMK_subsets, "GZMKCytotoxic_CD4_vs_CD8_subsets.csv",quote=FALSE)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 15, repr.plot.res = 100)

EnhancedVolcano(res_GZMK_subsets,
    lab = rownames(res_GZMK_subsets),
    x = 'log2FoldChange',
    y = 'padj',
    title = 'Cytotoxic 1 (GZMK CD8) versus Cytotoxic 2 (GZMK CD4)',
    #pCutoff = 10e-32,
    FCcutoff = 1.3,
    pointSize = 3.0,
    labSize = 6.0)
###pdf( "gzmk_cd8vcd4_volcano.pdf", width = 6, height = 10 )
